# Langchain

In [ ]:
import requests
from langchain_community.chat_models import ChatOllama
from langchain.agents import initialize_agent, Tool, AgentType
from datetime import datetime

In [68]:
# Simulation d'un appelle à une API météo
def get_weather(city):
    temp = 25
    wind = 30
    return f"Il fait {temp}°C avec un vent à {wind} km/h."

def get_time():
    return datetime.now().strftime("%H:%M:%S")

In [69]:
# On crée le Tool LangChain
weather_tool = Tool.from_function(
    func=get_weather,
    name="get_weather",
    description="Donne la météo actuelle pour une ville spécifiée."
)

# On instancie ton modèle Ollama
#llm = ChatOllama(model="gemma3n:e2b")
#llm = ChatOllama(model="qwen2.5:3b")
llm = ChatOllama(model="phi4-mini:latest")

# On initialise l'agent
agent = initialize_agent(
    tools=[weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prefix": """Si tu peux répondre directement à la question (sans appeler un outil), fais-le. 
N’utilise un outil que si c’est strictement nécessaire. 
Respecte scrupuleusement ce format pour les outils :

Thought: Raisonne brièvement
Action: nom_de_l_outil
Action Input: valeur"""}
)

# On pose une question à l'agent
result = agent.run("Peux-tu me dire la météo actuelle à Paris ?")
print(result)



> Entering new AgentExecutor chain...
Thought: The user is asking for current weather information in a specific location, which requires using an appropriate tool.
Action: get_weather
Action Input: city=Paris
Observation: Il fait 25°C avec un vent à 30 km/h.
Thought:Thought: I now have the requested weather details to provide as my final answer.

Final Answer: La météo actuelle à Paris est de 25°C avec un vent à une vitesse de 30 km/h.

> Finished chain.
La météo actuelle à Paris est de 25°C avec un vent à une vitesse de 30 km/h.


In [70]:
result = agent.run("Combien font 2+2 ?")
print(result)



> Entering new AgentExecutor chain...
The calculation "2 + 2" is a basic arithmetic problem with an objective and unambiguous correct numerical result that does not require any tools or external data sources. I can provide this simple mathematical fact directly.

Final Answer: The sum of 2 plus 2 equals 4.

> Finished chain.
The sum of 2 plus 2 equals 4.


Analyse :
Beaucoup de problémes soit il utilise trop l'outils soit il y a des erreurs de parsing. Pas très stable. Ou alors la réponse est en anglais.

# A la main

In [ ]:
import ollama
import re
import json

In [100]:
available_tools = {
    "get_weather": {
        "description": "Donne la météo actuelle pour une ville spécifiée.",
        "format": """{"name": "get_weather", "params": ["ville"]}""",
        "tool_function": get_weather
    },
    "get_time": {
        "description": "Donne l'heure actuelle.",
        "format": """{"name": "get_time", "params": []}""",
        "tool_function": get_time
    },
}

In [102]:
def ollama_llm_tool(question, available_tools):
    tools_description = ""
    for tool_name, tool_info in available_tools.items():
        tools_description += f"- Nom: {tool_name}\n"
        tools_description += f"  Description: {tool_info['description']}\n"
        tools_description += f"  Format: {tool_info['format']}\n\n"
    
    formatted_prompt = f"""Tu es un assistant intelligent qui doit analyser les questions des utilisateurs et déterminer si elles nécessitent l'utilisation d'un outil spécifique.

OUTILS DISPONIBLES:
{tools_description}

QUESTION DE L'UTILISATEUR:
{question}

    INSTRUCTIONS:
    1. Analyse la question de l'utilisateur
    2. Détermine si elle nécessite l'utilisation d'un des outils disponibles
    3. Réponds UNIQUEMENT par:
    - Format JSON: {{"tools": [{{"name": "nom_outil1", "params": ["paramètre1", "paramètre2",...]}},{{"name": "nom_outil2", "params": ["paramètre1",...]}}]}} si des outils sont nécessaires
    - Format JSON: {{"tools": "NONE"}} si tu peux répondre directement sans outil

    EXEMPLES:
    - Question: "Quel temps fait-il à Paris ?" → Réponse: {{"tools": [{{"name": "get_weather", "params": ["Paris"]}}]}}
    - Question: "Quelle heure est-il ?" → Réponse: {{"tools": [{{"name": "get_time", "params": []}}]}}
    - Question: "Dis-moi l'heure et la météo à Lyon" → Réponse: {{"tools": [{{"name": "get_time", "params": []}}, {{"name": "get_weather", "params": ["Lyon"]}}]}}
    - Question: "Combien font 2+2 ?" → Réponse: {{"tools": "NONE"}}

    RÉPONSE:"""
    
    response = ollama.chat(
        model="gemma3n:e2b",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    
    return final_answer

In [103]:
r1 = ollama_llm_tool("Peux-tu me dire la météo actuelle à Tallard et l'heur actuelle ?", available_tools)
r1

'{"tools": [{"name": "get_weather", "params": ["Tallard"]},{"name": "get_time", "params": []}]}'

In [104]:
r2 = ollama_llm_tool("Tu est plutot chien ou chat ?", available_tools)
r2

'{"tools": "NONE"}'

In [ ]:
def parse_tool_response(response):
    first_brace = response.find('{')
    last_brace = response.rfind('}')

    # Si on trouve les deux accolades
    if first_brace != -1 and last_brace != -1 and first_brace < last_brace:
        json_str = response[first_brace:last_brace + 1]
        print(f"JSON extrait: {json_str}")
        
        try:
            # Tenter de parser le JSON
            parsed = json.loads(json_str)
            print(f"JSON parsé avec succès: {parsed}")
        
        # Si il y a une erreur pour ne pas couper le programme on return {'tools': 'NONE'}
        except json.JSONDecodeError as e:
            print(f"Erreur de parsing JSON: {e}")
            return {'tools': 'NONE'}

    return parsed


In [143]:
tools_to_call = parse_tool_response(r1)
tools_to_call

JSON extrait: {"tools": [{"name": "get_weather", "params": ["Tallard"]},{"name": "get_time", "params": []}]}
JSON parsé avec succès: {'tools': [{'name': 'get_weather', 'params': ['Tallard']}, {'name': 'get_time', 'params': []}]}


{'tools': [{'name': 'get_weather', 'params': ['Tallard']},
  {'name': 'get_time', 'params': []}]}

In [144]:
def execute_tools(tools_to_call, available_tools):
    if tools_to_call.get("tools") == "NONE":
        return None

    results = []
    for tool in tools_to_call.get("tools", []):
        tool_name = tool["name"]
        params = tool["params"]
        
        if tool_name in available_tools:
            tool_info = available_tools[tool_name]
            tool_function = tool_info["tool_function"]
            tool_description = tool_info["description"]
            
            result = tool_function(*params)
            results.append({
                "tool_name": tool_name,
                "description": tool_description,
                "params": params,
                "result": result
            })
        else:
            print(f"Outil {tool_name} non reconnu.")
            results.append({
                "tool_name": tool_name,
                "description": "Outil non trouvé",
                "params": params,
                "result": f"Erreur: Outil '{tool_name}' non disponible"
            })
    
    return results

In [145]:
execute_tools(tools_to_call, available_tools)

[{'tool_name': 'get_weather',
  'description': 'Donne la météo actuelle pour une ville spécifiée.',
  'params': ['Tallard'],
  'result': 'Il fait 25°C avec un vent à 30\u202fkm/h.'},
 {'tool_name': 'get_time',
  'description': "Donne l'heure actuelle.",
  'params': [],
  'result': '11:50:13'}]

In [158]:
def run_llm_agent(question, available_tools):

    tools_to_call = parse_tool_response(ollama_llm_tool(question, available_tools))
    tools_results = execute_tools(tools_to_call, available_tools)

    if tools_results is None:
        formatted_prompt = f"""Tu es un assistant intelligent. Réponds à cette question :

QUESTION: {question}

Réponds de manière naturelle et conversationnelle en français."""
    
    else:
        # Formatter les résultats des outils
        tools_summary = ""
        for tool_result in tools_results:
            tool_name = tool_result["tool_name"]
            description = tool_result["description"]
            params = tool_result["params"]
            result = tool_result["result"]
            
            tools_summary += f"- Outil: {tool_name} ({description})\n"
            tools_summary += f"  Paramètres: {params}\n"
            tools_summary += f"  Résultat: {result}\n\n"
        
        formatted_prompt = f"""Tu es un assistant intelligent. L'utilisateur a posé une question et j'ai utilisé des outils pour obtenir des informations.

QUESTION DE L'UTILISATEUR:
{question}

RÉSULTATS DES OUTILS:
{tools_summary}

INSTRUCTIONS:
- Utilise les résultats des outils pour répondre à la question de l'utilisateur
- Réponds de manière naturelle et conversationnelle en français
- Synthétise les informations de manière claire et utile
- Ne mentionne pas les noms techniques des outils, parle naturellement

RÉPONSE:"""
    response = ollama.chat(
        model="gemma3n:e2b",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    
    return final_answer

In [160]:
run_llm_agent("Combien d'habitant possède la france ?", available_tools)

JSON extrait: {"tools": "NONE"}
JSON parsé avec succès: {'tools': 'NONE'}


"Salut ! 😊\n\nLa France a environ 68 millions d'habitants en ce moment. C'est une population assez importante, et elle est répartie sur tout le territoire ! \n\nTu veux savoir quelque chose d'autre sur la France ? Par exemple, la population des grandes villes, ou peut-être la répartition géographique ? Je peux essayer de t'aider ! 😉"

Résultat : Bien meilleur ! Même s'il n'y a pas de callback à répétition comme avec LangChain, le modèle tourne moins en boucle et se concentre davantage sur ses tâches. Plus précis, en tout cas avec un petit modèle.